In [4]:
import json
from comet_ml import Experiment

import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav

from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

In [9]:
with open('config.json') as f:
    config = json.load(f)
    comet_config = config['comet']

In [10]:
# Initialize Comet experiment
experiment = Experiment(api_key=comet_config['apiKey'], workspace=comet_config['workSpace'], project_name=comet_config["projectName"])

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/kpapijnenburg/urban-sound-classifier/51a33934f25447fb9e8421a93e40719f



In [12]:
# Load dataset
df = pd.read_csv('UrbanSound8k/metadata/UrbanSound8k.csv')
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing
